# Have a first look at the trajectory data

## Technical preamble

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

import intake  # data catalogs

import xarray as xr  # provides basic datatypes for labeled nd arrays

## Set parameters

Let's make sure that this is the only place where users have to change contents.

In [ ]:
# parameters

platform = "gke"

## Open the data catalog

In [ ]:
catalog = intake.open_catalog(f"intake-catalogs/medseaconnectivity_{platform}.yaml")
print(list(catalog))

## Load a dataset

There's two sets of trajectories:
- `'medsea-trajectories-stokes'` contains trajectories at the sea surface that are feeling to drift by waves and the Ocean circulation.
- `'medsea-trajectories'` contains trajectories at the sea surface and in deeper layers that are only feeling the Ocean circulation.

Here, we'll only use the smaller `'medsea-trajectories-stokes'` dataset. Let's load it:

In [ ]:
dataset = catalog['medsea-trajectories-stokes'].to_dask()
display(dataset)
print(f"amounts to {dataset.nbytes / 1e9} GB")

## Restrict to subset of the data

For now, we'll only use the first 50.000 trajectories. And we'll load all of it to memory.

In [ ]:
dataset = dataset.isel(traj=slice(0, 50_000))
dataset = dataset.load()
display(dataset)
print(f"amounts to {dataset.nbytes / 1e9} GB")

## Inspect the data

There's two dimensions:
- `traj` numbers individual trajectories
- `obs` numbers time steps along each trajectory

And there's data variables. For now, we'll only deal with a few:
- `lat` gives the latitude of each trajectory at each observation.
- `lon`: gives the longitude of each trajectory at each observation.
- `temp`: gives the temperature of each trajectory at each observation.
- `time`: gives the time stamp of each trajectory at each observation.

## Plot larvae positions with matplotlib

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(7, 7), sharex=True, sharey=True)
ax = ax.flatten()

for hours, current_ax in zip([0, 10, 100, 961], ax):
    dataset.isel(obs=hours).plot.scatter(x="lon", y="lat", s=10, alpha=0.01, ax=current_ax);
    current_ax.set_title(f"after {hours} hours");

    fig.tight_layout();

## Plot larvae positions with holoviews and datashader

In [ ]:
import holoviews as hv
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.operation import decimate
hv.extension('bokeh')

In [ ]:
points = hv.Points(
    (dataset.lon.data.flat, dataset.lat.data.flat)
)

In [ ]:
datashade(points)

## Plot on a map

In [ ]:
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs

In [ ]:
points_geo = gv.Points(
    (dataset.lon.data.flat, dataset.lat.data.flat)
)

In [ ]:
(
    datashade(points_geo) * gf.coastline * gf.grid * gf.borders
).opts(
    'Feature', projection=crs.Geostationary(), global_extent=False,
    width=400, height=400
)